In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)

### CSCI 447 Fall 2023 Kaggle Competition

In [7]:
train = pd.read_csv('data/kaggle_train.csv')
test = pd.read_csv('data/kaggle_test.csv')
# example = pd.read_csv('data/kaggle_example_submission.csv')

In [ ]:
train

In [ ]:
# show features that correlate most with target
train.corr()['target'].sort_values(ascending=False)

---

In [ ]:
# # Remove features that correlate weakly with target (run this to do that)
# train_correlated = train.copy()
# train_correlated = train_correlated.drop(['f14', 'f13','f11'], axis=1)

# train = train_correlated

In [8]:
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV

In [9]:
targets = train.target
train_ids = train.id
test_ids = test.id
# standardize data with mean 0 and std 1
train_mean = train.mean()
train_std = train.std()
train = (train - train_mean) / train_std
test = (test - train_mean) / train_std
# add target, ids back
train['target'] = targets
train['id'] = train_ids
test['id'] = test_ids
train.describe()

,id,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,f18,f19,f20,target
count,400.000000,4.000000e+02,4.000000e+02,4.000000e+02,4.000000e+02,4.000000e+02,4.000000e+02,4.000000e+02,4.000000e+02,4.000000e+02,4.000000e+02,4.000000e+02,4.000000e+02,4.000000e+02,4.000000e+02,4.000000e+02,4.000000e+02,4.000000e+02,4.000000e+02,4.000000e+02,4.000000e+02,400.000000
mean,200.500000,-2.353673e-16,-8.881784e-18,-8.881784e-17,4.440892e-18,1.440092e-13,3.463896e-15,1.421085e-16,7.549517e-17,1.998401e-16,-4.884981e-17,-3.108624e-17,4.440892e-17,1.776357e-17,-4.440892e-18,-3.463896e-16,4.440892e-17,2.198242e-16,-4.440892e-18,3.375078e-16,-1.332268e-17,-1.130048
std,115.614301,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,10.848223
min,1.000000,-2.861330e+00,-2.908670e+00,-3.173200e+00,-2.869726e+00,-2.672945e+00,-2.720882e+00,-2.989203e+00,-3.012330e+00,-2.965846e+00,-2.685398e+00,-2.635389e+00,-3.054029e+00,-2.627419e+00,-3.272950e+00,-3.106918e+00,-2.831742e+00,-2.779386e+00,-3.636015e+00,-3.246809e+00,-2.792469e+00,-56.422845
25%,100.750000,-6.225042e-01,-6.692259e-01,-6.522357e-01,-6.899493e-01,-6.424957e-01,-6.776854e-01,-6.741393e-01,-7.111297e-01,-6.773207e-01,-6.629881e-01,-6.949257e-01,-6.308606e-01,-6.886025e-01,-6.904504e-01,-6.458838e-01,-7.728394e-01,-6.576152e-01,-6.672167e-01,-6.808850e-01,-7.552145e-01,-5.335921
50%,200.500000,8.652263e-02,-1.645857e-04,2.338077e-02,-8.374797e-02,3.323209e-02,-4.310863e-02,-3.067301e-02,4.532217e-02,-6.641827e-02,-8.207512e-03,-4.218802e-02,2.704563e-02,2.208377e-02,1.830090e-02,-7.686029e-02,-3.696830e-02,1.885444e-02,-5.770789e-03,4.207513e-02,-3.479021e-02,-0.510613
75%,300.250000,6.244502e-01,6.850371e-01,6.551474e-01,6.638246e-01,7.158002e-01,7.040773e-01,6.506110e-01,7.178836e-01,6.909541e-01,6.868708e-01,6.867154e-01,6.994259e-01,6.030659e-01,7.090012e-01,6.896613e-01,7.284371e-01,6.721913e-01,6.771813e-01,6.989213e-01,6.907257e-01,3.934202
max,400.000000,2.947224e+00,2.515782e+00,2.758949e+00,2.681736e+00,3.034223e+00,2.618329e+00,2.843750e+00,2.735974e+00,2.822296e+00,3.395438e+00,2.839666e+00,2.865029e+00,3.017816e+00,2.529155e+00,2.749985e+00,3.094787e+00,3.136531e+00,2.993225e+00,2.461094e+00,2.694553e+00,36.101080


#### XGBoost:

In [10]:
# split data into train and test sets
X = train.drop(['target', 'id'], axis=1)
y = train['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

In [11]:
xgb = XGBRegressor(random_state=42)
xgb.fit(X_train, y_train)
y_pred = xgb.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print('RMSE:', np.sqrt(mse))

RMSE: 5.083759023990873


In [ ]:
# # grid search on xgb
# from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

# params = {
#     'n_estimators': [100, 200, 300, 400, 500],
#     'learning_rate': [0.01, 0.1, 0.2],
#     'max_depth': [1, 2, 3, 4, 5],
#     'min_child_weight': [1, 2, 3],
#     'gamma': [0, 0.1, 0.2],
#     'subsample': [0.5, 0.75, 1],
#     'colsample_bytree': [0.5, 0.75, 1],
#     'reg_alpha': [0, 0.1, 0.2],
#     'reg_lambda': [0, 0.1, 0.2]
# }
# grid = GridSearchCV(estimator=xgb, param_grid=params, scoring='neg_mean_squared_error', cv=5, verbose=1)
# # grid = RandomizedSearchCV(estimator=xgb, param_distributions=params, scoring='neg_mean_squared_error', cv=5, verbose=1)
# grid.fit(X_train, y_train)
# print(grid.best_params_)
# print(grid.best_score_)
# best_xgb = grid.best_estimator_
# y_pred = best_xgb.predict(X_test)
# mse = mean_squared_error(y_test, y_pred)
# print('MSE:', mse)

In [12]:
xgb_best_so_far = XGBRegressor(n_estimators=500, learning_rate=0.2, max_depth=1, min_child_weight=1, gamma=0.1, random_state=42)
xgb_best_so_far.fit(X_train, y_train)
y_pred = xgb_best_so_far.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print('RMSE:', np.sqrt(mse))

RMSE: 2.421711100857374


#### FEATURE ENGINEERING:

In [ ]:
# find most important features
from xgboost import plot_importance
plot_importance(xgb_best_so_far)
plt.show()

In [13]:
train_reduced = train[['f4','f5','f1','f3','target']]
X = train_reduced.drop(['target'], axis=1)
y = train_reduced['target']

X_train_reduced, X_test_reduced, y_train_reduced, y_test_reduced = train_test_split(X, y, test_size=0.2, random_state=42)
xgb_best_so_far.fit(X_train_reduced, y_train_reduced)
y_pred = xgb_best_so_far.predict(X_test_reduced)
mse = mean_squared_error(y_test_reduced, y_pred)
print('RMSE:', np.sqrt(mse))
print('MSE:', mse)

RMSE: 1.9698885792473488
MSE: 3.8804610146491383


In [ ]:
# plot predictions vs actual
plt.scatter(y_test_reduced, y_pred)
plt.xlabel('Actual')
plt.ylabel('Predicted')
plt.show()

In [18]:
# predict test set and save to csv with ID, target
test_reduced = test[['id', 'f4','f5','f1','f3']]
test_reduced = test_reduced.drop(['id'], axis=1)
predictions = xgb_best_so_far.predict(test_reduced)
predictions = pd.DataFrame(predictions, columns=['target'])
predictions = pd.concat([test, predictions], axis=1)
# predictions = predictions[['target']]
predictions = predictions[['id', 'target']]
predictions

,id,target,target
0,401,NaN,-36.986443
1,402,NaN,-7.262199
2,403,NaN,-0.516774
3,404,NaN,-4.730848
4,405,NaN,3.133424
...,...,...,...
795,1196,NaN,1.107090
796,1197,NaN,1.066249
797,1198,NaN,24.955490
798,1199,NaN,-3.751294


In [17]:
# drop the 2nd column using its id
predictions = predictions.drop([predictions.index[1]])
predictions

,id,target,target
0,401,NaN,-36.986443
2,403,NaN,-0.516774
3,404,NaN,-4.730848
4,405,NaN,3.133424
5,406,NaN,-15.546897
...,...,...,...
795,1196,NaN,1.107090
796,1197,NaN,1.066249
797,1198,NaN,24.955490
798,1199,NaN,-3.751294


In [ ]:
predictions.to_csv('data/kaggle_submission_bova_3.csv', index=False)

---

## Neural Network

In [ ]:
# # tensorflow regression neural network
# import tensorflow as tf
# from tensorflow import keras
# from tensorflow.keras import layers
# from tensorflow.keras import regularizers

# # split data into train and test sets
# X = train.drop(['target', 'id'], axis=1)
# y = train['target']
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

# model = keras.Sequential([
#     layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
#     layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
#     layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
#     layers.Dense(1)
# ])

# model.compile(optimizer='adam', loss='mse', metrics=['mae'])
# history = model.fit(X_train, y_train, validation_split=0.2, epochs=100, verbose=1)

# # plot mse
# plt.plot(history.history['loss'])
# plt.plot(history.history['val_loss'])
# plt.title('Model MSE')
# plt.ylabel('MSE')
# plt.xlabel('Epoch')
# plt.legend(['Train', 'Validation'], loc='upper right')
# plt.show()